In [1]:
# Reading the input data
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


#### Reading and using GFVs generated by AE

In [2]:
# Split the data into train and test sets
import os
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.decomposition import PCA

drive_root = '/content/drive/MyDrive/RLFinalProjectFiles'
org_root = os.path.join(drive_root, 'shape_net_core_uniform_samples_2048')

del_ratio = 50

import numpy as np
from random import random

def read_gfv_data(file_path):
    gfvs = np.load(file_path)
    return gfvs

# Example usage
directory = drive_root + "/gfv_vae3/50/"
def read_all_gfv_data(directory):
    gfvs = []
    for filename in os.listdir(directory):
        if filename.endswith(".npy"):
            file_path = os.path.join(directory, filename)
            gfvs.extend(read_gfv_data(file_path))
    return gfvs

from sklearn.decomposition import KernelPCA

gfvs = read_all_gfv_data(directory)
print(len(gfvs))
selected_gfvs = gfvs[:2500]
gfvs_tensor = torch.tensor(np.array(selected_gfvs))

# Step 4: Apply Kernel PCA
# kpca = KernelPCA(n_components=256, kernel='rbf')
# transformed_data = kpca.fit_transform(gfvs_tensor.numpy())

# # Step 5: Get number of components
# # num_components = kpca.n_components_

# # Print shape of data
# print(transformed_data.shape)
print(gfvs[0].shape)

# Reshape the tensor to [N, C, W, H] assuming the shape of each GFV is already [C, W, H]
# If not, adjust the shape accordingly
gfvs_tensor = torch.tensor(gfvs_tensor).unsqueeze(2)  # Add a channel dimension assuming it's missing
# gfvs_tensor = gfvs_tensor.permute(0, 3, 1, 2)  # Permute dimensions to [N, C, W, H]
print(gfvs_tensor.shape)

if gfvs is not None:
    print("GFV data shape:", gfvs_tensor.shape)
else:
    print("Failed to read GFV data.")




2609
(256,)
torch.Size([2500, 256, 1])
GFV data shape: torch.Size([2500, 256, 1])


<ipython-input-2-9ab4b9b96f8e>:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  gfvs_tensor = torch.tensor(gfvs_tensor).unsqueeze(2)  # Add a channel dimension assuming it's missing


#### Autoencoder for feature dimension reduction for encoder output

#### GAN architecture

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

class SelfAttention(nn.Module):
    """Self Attention Layer."""

    def __init__(self, in_dim):
        super(SelfAttention, self).__init__()

        self.query_conv = nn.Conv2d(in_channels=in_dim, out_channels=in_dim//8, kernel_size=1)
        self.key_conv = nn.Conv2d(in_channels=in_dim, out_channels=in_dim//8, kernel_size=1)
        self.value_conv = nn.Conv2d(in_channels=in_dim, out_channels=in_dim, kernel_size=1)
        self.gamma = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        batch, channels, height, width = x.size()

        proj_query = self.query_conv(x).view(batch, -1, height*width).permute(0, 2, 1)
        proj_key = self.key_conv(x).view(batch, -1, height*width)

        energy = torch.bmm(proj_query, proj_key)
        attention = F.softmax(energy, dim=-1)

        proj_value = self.value_conv(x).view(batch, -1, height*width)

        out = torch.bmm(proj_value, attention.permute(0, 2, 1))
        out = out.view(batch, channels, height, width)

        out = self.gamma * out + x
        return out

class Generator(nn.Module):
    """Generator."""

    def __init__(self, z_dim=1, gfvs_dim=256, conv_dim=64):
        super(Generator, self).__init__()
        self.gfvs_dim = gfvs_dim

        # Calculate the initial image size based on the GFVs shape
        self.initial_image_size = int(np.sqrt(gfvs_dim / conv_dim / 4))

        # Adjust the input dimension to match the latent vector size
        self.fc = nn.Linear(z_dim, conv_dim * 8 * self.initial_image_size * self.initial_image_size)

        # Define the convolutional layers
        self.conv1 = nn.ConvTranspose2d(conv_dim * 8, conv_dim * 4, 4, 2, 1)
        self.sa1 = SelfAttention(conv_dim * 4)
        self.conv2 = nn.ConvTranspose2d(conv_dim * 4, conv_dim * 2, 4, 2, 1)
        self.sa2 = SelfAttention(conv_dim * 2)
        self.conv3 = nn.ConvTranspose2d(conv_dim * 2, conv_dim, 4, 2, 1)
        self.sa3 = SelfAttention(conv_dim)
        self.conv4 = nn.ConvTranspose2d(conv_dim , 1, 4, 2, 1)  # Output should match GFVs size
        self.fc_output = nn.Linear(conv_dim * self.initial_image_size * self.initial_image_size, gfvs_dim)

    def forward(self, z):
        out = self.fc(z)
        out = out.view(-1, 64 * 8, self.initial_image_size, self.initial_image_size)
        out = F.relu(self.conv1(out))
        out = self.sa1(out)
        out = F.relu(self.conv2(out))
        out = self.sa2(out)
        out = F.relu(self.conv3(out))
        out = self.sa3(out)
        out = torch.tanh(self.conv4(out))  # Apply tanh activation for image generation
        # out = out.squeeze()  # Flatten the output
        out = out.permute(0, 2, 3, 1)
        out = out.view(-1, 64 * self.initial_image_size * self.initial_image_size) # Reshape to match GFVs dimension
        return out





class Discriminator(nn.Module):
    """Discriminator."""

    def __init__(self, gfvs_dim=256, conv_dim=64):
        super(Discriminator, self).__init__()
        self.gfvs_dim = gfvs_dim

        # Calculate the final image size based on the GFVs shape
        self.final_image_size = int(np.sqrt(gfvs_dim/ conv_dim / 4))

        # Adjust the input dimension to match the GFVs size
        self.conv1 = nn.Conv2d(1, conv_dim, 1)
        self.conv2 = nn.Conv2d(conv_dim, conv_dim * 2, 3, 2, 1)
        self.sa1 = SelfAttention(conv_dim * 2)
        self.conv3 = nn.Conv2d(conv_dim * 2, conv_dim * 4, 3, 2, 1)
        self.sa2 = SelfAttention(conv_dim * 4)
        self.conv4 = nn.Conv2d(conv_dim * 4, 1, 1)

    def forward(self, x):
        out = F.leaky_relu(self.conv1(x), 0.2)
        out = F.leaky_relu(self.conv2(out), 0.2)
        out = self.sa1(out)
        out = F.leaky_relu(self.conv3(out), 0.2)
        out = self.sa2(out)
        out = self.conv4(out)
        return out.squeeze()  # Return a 1D output


#### Chamfer Loss

In [6]:
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 30.7 MB/s eta 0:00:00


In [10]:
import torch
import numpy as np
import faiss
import torch.nn as nn


def robust_norm(var):
    '''
    :param var: Variable of BxCxHxW
    :return: p-norm of BxCxW
    '''
    result = ((var**2).sum(dim=2) + 1e-8).sqrt()
    # result = (var ** 2).sum(dim=2)

    # try to make the points less dense, caused by the backward loss
    # result = result.clamp(min=7e-3, max=None)
    return result

class ChamferLoss(nn.Module):
    def __init__(self, opt):
        super(ChamferLoss, self).__init__()
        self.dimension = 3
        self.k = 1

        # we need only a StandardGpuResources per GPU
        # self.res = faiss.StandardGpuResources()
        # self.res.setTempMemoryFraction(0.1)

        # place holder
        self.forward_loss = torch.FloatTensor([0])
        self.backward_loss = torch.FloatTensor([0])

    def build_nn_index(self, database):
        '''
        :param database: numpy array of Nx3
        :return: Faiss index, in CPU
        '''
        database = np.ascontiguousarray(database)
        index = faiss.IndexFlatL2(self.dimension)
        index.add(database)
        return index

    def search_nn(self, index, query, k):
        '''
        :param index: Faiss index
        :param query: numpy array of Nx3
        :return: D: Variable of Nxk, type FloatTensor, in CPU
                 I: Variable of Nxk, type LongTensor, in CPU
        '''
        D, I = index.search(query, k)

        D_var = torch.from_numpy(np.ascontiguousarray(D))
        I_var = torch.from_numpy(np.ascontiguousarray(I).astype(np.int64))

        return D_var, I_var

    def forward(self, predict_pc, gt_pc):
        '''
        :param predict_pc: Bx3xM Variable in CPU
        :param gt_pc: Bx3xN Variable in CPU
        :return:
        '''
        # for VAE
        # predict_pc = predict_pc[0]
        predict_pc_size = predict_pc.size()
        gt_pc_size = gt_pc.size()

        # print(predict_pc.shape)

        predict_pc_np = np.ascontiguousarray(torch.transpose(predict_pc.data.clone(), 1, 2).numpy())  # BxMx3
        gt_pc_np = np.ascontiguousarray(torch.transpose(gt_pc.data.clone(), 1, 2).numpy())  # BxNx3

        # selected_gt: Bxkx3xM
        selected_gt_by_predict = torch.FloatTensor(predict_pc_size[0], self.k, predict_pc_size[1], predict_pc_size[2])
        # selected_predict: Bxkx3xN
        selected_predict_by_gt = torch.FloatTensor(gt_pc_size[0], self.k, gt_pc_size[1], gt_pc_size[2])

        # process each batch independently.
        for i in range(predict_pc_np.shape[0]):
            index_predict = self.build_nn_index(predict_pc_np[i])
            index_gt = self.build_nn_index(gt_pc_np[i])

            # database is gt_pc, predict_pc -> gt_pc -----------------------------------------------------------
            _, I_var = self.search_nn(index_gt, predict_pc_np[i], self.k)

            # process nearest k neighbors
            for k in range(self.k):
                selected_gt_by_predict[i,k,...] = gt_pc[i].index_select(1, I_var[:,k])

            # database is predict_pc, gt_pc -> predict_pc -------------------------------------------------------
            _, I_var = self.search_nn(index_predict, gt_pc_np[i], self.k)

            # process nearest k neighbors
            for k in range(self.k):
                selected_predict_by_gt[i,k,...] = predict_pc[i].index_select(1, I_var[:,k])

        # compute loss ===================================================
        # selected_gt(Bxkx3xM) vs predict_pc(Bx3xM)
        forward_loss_element = robust_norm(selected_gt_by_predict-predict_pc.unsqueeze(1).expand_as(selected_gt_by_predict))
        self.forward_loss = forward_loss_element.mean()
        self.forward_loss_array = forward_loss_element.mean(dim=1).mean(dim=1)

        # selected_predict(Bxkx3xN) vs gt_pc(Bx3xN)
        backward_loss_element = robust_norm(selected_predict_by_gt - gt_pc.unsqueeze(1).expand_as(selected_predict_by_gt))  # BxkxN
        self.backward_loss = backward_loss_element.mean()
        self.backward_loss_array = backward_loss_element.mean(dim=1).mean(dim=1)

        # self.loss_array = self.forward_loss_array + self.backward_loss_array
        return self.forward_loss + self.backward_loss

    def __call__(self, predict_pc, gt_pc):
        # start_time = time.time()
        loss = self.forward(predict_pc, gt_pc)
        # print(time.time()-start_time)
        return loss


#### Training GAN

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
from torch.autograd import Variable

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
z_dim = 1
pc_dim = 256
batch_size = 50
lr = 0.01
num_epochs = 10
lambda_gp = 10  # Gradient penalty coefficient

# Initialize generator and discriminator
generator = Generator(z_dim=z_dim, gfvs_dim=pc_dim, conv_dim=64)
discriminator = Discriminator(gfvs_dim=pc_dim)

chamfer_criterion = ChamferLoss({})
# Optimizers
optimizer_G = optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.9))
optimizer_D = optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.9))

# Dummy GFV data (replace with your actual GFV data)
# num_samples = 1000
gfvs_real_tensor = torch.tensor(selected_gfvs)
gfvs_real = gfvs_real_tensor

# Data loader
dataset = TensorDataset(gfvs_real)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Gradient penalty calculation
def compute_gradient_penalty(discriminator, real_samples, fake_samples):
    alpha = torch.rand(batch_size, 1, 1, 1, device=device)

    # Ensure the fake_samples have the same shape as real_samples
    real_samples = real_samples.unsqueeze(0)
    fake_samples = fake_samples.view(1, real_samples.size(1), real_samples.size(2))
    interpolated = alpha * real_samples + (1 - alpha) * fake_samples
    interpolated.requires_grad_(True)
    prob_interpolated = discriminator(interpolated)
    gradients = autograd.grad(outputs=prob_interpolated, inputs=interpolated,
                              grad_outputs=torch.ones(prob_interpolated.size(), device=device),
                              create_graph=True, retain_graph=True)[0]
    gradients_norm = torch.sqrt(torch.sum(gradients ** 2, dim=(1, 2, 3)))
    gradient_penalty = ((gradients_norm - 1) ** 2).mean()
    return gradient_penalty

# Training loop
for epoch in range(num_epochs):
    for gfvs_batch in tqdm(dataloader):
        # Move batch to device
        gfvs_real_batch = gfvs_batch[0]
        gfvs_real_batch_tensor = torch.stack(gfvs_batch)
        # print("gfvs_batch ",gfvs_real_batch_tensor.shape)
        gfvs_real_batch_tensor = gfvs_real_batch_tensor.view(batch_size,256,1)


        # Train Discriminator
        optimizer_D.zero_grad()
        gfvs_real_batch_tensor = gfvs_real_batch_tensor.unsqueeze(1)
        # Real batch
        real_output = discriminator(gfvs_real_batch_tensor)
        d_loss_real = -real_output.mean()

        # Fake batch
        fixed_z = torch.rand(batch_size, z_dim)  # Generate random values between 0 and 1
        fake_gfvs = generator(fixed_z)
        fake_gfvs = fake_gfvs.unsqueeze(0).unsqueeze(1)
        fake_output = discriminator(fake_gfvs.detach())
        d_loss_fake = fake_output.mean()

        # Gradient penalty
        gradient_penalty = compute_gradient_penalty(discriminator, gfvs_real_batch, fake_gfvs)

        # Total loss
        d_loss = d_loss_fake + d_loss_real + lambda_gp * gradient_penalty
        d_loss.backward()
        optimizer_D.step()

        # Train Generator
        optimizer_G.zero_grad()
        z = torch.randn(gfvs_real_batch.size(0), z_dim, device=device)
        fake_gfvs = generator(z)
        fake_gfvs = fake_gfvs.view(batch_size, 1, 16, 16)
        # print("fake_gfvs shape ",fake_gfvs.shape)
        fake_output = discriminator(fake_gfvs)
        g_loss_chamfer = chamfer_criterion(fake_gfvs, gfvs_real_batch.transpose(1, 2))
        g_loss_gfv = torch.norm(fake_gfvs - gfvs_real_batch, p=2)
        g_loss = g_loss_chamfer + g_loss_gfv
        g_loss.backward()
        optimizer_G.step()

    # Print losses
    print(f"Epoch [{epoch + 1}/{num_epochs}], Discriminator Loss: {d_loss.item()}, Generator Loss: {g_loss.item()}")

# Save model
torch.save(generator.state_dict(), drive_root + "/models/generator2.pth")
torch.save(discriminator.state_dict(), drive_root +"/models/discriminator2.pth")
